## Data preprocessing

### Load libraries

In [ ]:
# Librerie base
import numpy as np
import pandas as pd

# Visualizzazione
import matplotlib.pyplot as plt  
import seaborn as sns

# Modelli statistici
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm

# Libreria ISLP (Statistical Learning)
from ISLP import load_data
from ISLP.models import (ModelSpec as MS ,summarize, poly)

We will use the Aircraft Price dataset, from Kaggle. 

### Dataset load 

In [ ]:
# carico il dataset e stampo l'head
data = pd.read_csv("Data/aircraft_price.csv", encoding='utf-8')
target = 'price'

### data info

In [ ]:
data.columns

In [ ]:
print(data.describe())

In [ ]:
# Controllo i dati nulli
columnsWithNulls=data.isnull().sum().sort_values(ascending=False)
columnsWithNulls=columnsWithNulls[columnsWithNulls>0]
print(columnsWithNulls)

In [ ]:
# Controllo in percentuale quanti dati mancano così da capire come trattarli, pongo una soglia di eliminazione del regressore nel caso di +30% di dati mancanti
missing_pct = data.isnull().mean() * 100
missing_only = missing_pct[missing_pct > 0].sort_values(ascending=False)
print(missing_only )

In [ ]:
#Drop dati nulli
data.dropna(axis=0, inplace=True)
missing_pct = data.isnull().sum()
print("Number of null data: \n")
print(missing_pct)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
numeric_data = data.select_dtypes(include=[np.number])
# Handle missing or infinite values in the data
numeric_data_cleaned = numeric_data.replace([np.inf, -np.inf], np.nan).dropna()

numeric_data_with_const = sm.add_constant(numeric_data_cleaned)

# Calcola il VIF per ciascuna variabile
vif_data = pd.DataFrame()
vif_data["Regressor"] = numeric_data_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(numeric_data_with_const.values, i) for i in range(numeric_data_with_const.shape[1])]

vif_data = vif_data.sort_values("VIF", ascending=False)
print(vif_data)


### Skewness check

In [ ]:
# Plot distributions of all numeric variables
numeric_cols = data.select_dtypes(include=['number']).columns
n = len(numeric_cols)
cols = 4
rows = (n + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
for ax, col in zip(axes.flatten(), numeric_cols):
    sns.histplot(data[col], kde=True, ax=ax)
    ax.set_title(col)

# remove any unused subplots
for ax in axes.flatten()[n:]:
    ax.remove()

plt.tight_layout()
plt.show()

In [ ]:
#calcooliamo la skewness
skewness = data[numeric_cols].apply(lambda x: x.skew()).sort_values(ascending=False)
print("Skewness of numeric variables:")
print(skewness)


In [ ]:
# colonne da log-trasformare (skewness > 1)
skewed_feats = [
    'fuel_tank', 'engine_power', 'landing_distance', 'empty_weight',
    'all_eng_roc', 'range', 'wing_span', 'length',
    'out_eng_roc', 'max_speed', 'cruise_speed', 'takeoff_distance'
]

# copia del dataset e applicazione del log(1+x)
logData = data.copy()
for col in skewed_feats:
    logData[col] = np.log1p(logData[col])

# log-trasformartion anche del target:
logData['price'] = np.log1p(logData['price'])

# logData è ora il tuo dataset con le variabili selezionate log-trasformate


In [ ]:
#verifichiamo la skewness
# Plot distributions of all numeric variables
numeric_cols = logData.select_dtypes(include=['number']).columns
n = len(numeric_cols)
cols = 4
rows = (n + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
for ax, col in zip(axes.flatten(), numeric_cols):
    sns.histplot(logData[col], kde=True, ax=ax)
    ax.set_title(col)

# remove any unused subplots
for ax in axes.flatten()[n:]:
    ax.remove()

plt.tight_layout()
plt.show()

## log transformation of target 

In [ ]:
import numpy as np

## faccio log del target e confronto con target originale
target = 'price'
log_target = np.log(data[target])
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(data[target], bins=30, color='blue', alpha=0.7)
plt.title('Distribuzione originale del target')
plt.subplot(1, 2, 2)
plt.hist(log_target, bins=30, color='green', alpha=0.7)
plt.title('Distribuzione log del target')
plt.show()


#
- **Distribuzione originale** (istogramma blu): il prezzo è fortemente asimmetrico a destra, con code lunghe fino a 5 M€ e una concentrazione massima attorno a 1–3 M€. Questo livello di skewness può causare problemi di eteroschedasticità e di instabilità nella regressione OLS.  
- **Distribuzione log** (istogramma verde): dopo aver fatto `log(price)`, i valori si raggruppano su una curva molto più simile a una normale — la coda lunga si riduce, e l’intervallo diventa circa 13.5–15.5 sul logaritmo naturale.  

**Cosa ci dice?**  
1. **Assunti di OLS**: il log‐transform aiuta a soddisfare meglio l’ipotesi di normalità dei residui e di varianza costante.  
2. **Linearità percentuale**: un modello lineare su \(\log(price)\) interpreta i coefficienti come variazioni percentuali del prezzo, spesso più sensate di variazioni assolute su una scala ampia.  
3. **Robustezza**: riduci l’influenza degli outlier più estremi, migliorando stabilità e predittività.  


## Correlation matrix

In [ ]:
# Seleziona le colonne numeriche
numeric_data = data.select_dtypes(include=[np.number])
if numeric_data.shape[1] >= 4:
    # Calcola la matrice di correlazione
    corr = numeric_data.corr()
    
    # Imposta la figura
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        corr,
        annot=True,
        fmt='.2f',
        cmap='inferno_r',
        square=True,
        linewidths=.5,
        cbar_kws={"shrink": .75}
    )
    plt.title('Correlation Heatmap of Numeric Features')
    plt.tight_layout()
    plt.show()
else:
    print('Not enough numeric columns for a meaningful correlation heatmap')

In [ ]:
# Seleziona le colonne numeriche
numeric_data = logData.select_dtypes(include=[np.number])
if numeric_data.shape[1] >= 4:
    # Calcola la matrice di correlazione
    corr = numeric_data.corr()
    
    # Imposta la figura
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        corr,
        annot=True,
        fmt='.2f',
        cmap='inferno_r',
        square=True,
        linewidths=.5,
        cbar_kws={"shrink": .75}
    )
    plt.title('Correlation Heatmap of Numeric Features (After Log Transformation)')
    plt.tight_layout()
    plt.show()
else:
    print('Not enough numeric columns for a meaningful correlation heatmap')

## VIF 

### without log transformation

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Seleziona le colonne numeriche
numeric_data = data.select_dtypes(include=[np.number])
# Handle missing or infinite values in the data
numeric_data_cleaned = numeric_data.replace([np.inf, -np.inf], np.nan).dropna()
numeric_data_with_const = sm.add_constant(numeric_data_cleaned)
# Calcola il VIF per ciascuna variabile
vif_data = pd.DataFrame()
vif_data["Regressor"] = numeric_data_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(numeric_data_with_const.values, i) for i in range(numeric_data_with_const.shape[1])]
vif_data = vif_data.sort_values("VIF", ascending=False)
print(vif_data)

### with log transformation

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Seleziona le colonne numeriche
numeric_data = logData.select_dtypes(include=[np.number])
# Handle missing or infinite values in the data
numeric_data_cleaned = numeric_data.replace([np.inf, -np.inf], np.nan).dropna()
numeric_data_with_const = sm.add_constant(numeric_data_cleaned)
# Calcola il VIF per ciascuna variabile
vif_data = pd.DataFrame()
vif_data["Regressor"] = numeric_data_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(numeric_data_with_const.values, i) for i in range(numeric_data_with_const.shape[1])]
vif_data = vif_data.sort_values("VIF", ascending=False)
print(vif_data)

**Commento sui VIF**  
Dall’analisi dei VIF emerge innanzitutto che alcune coppie di variabili sono fortemente collineari (in particolare **landing_distance** ed **empty_weight**, con VIF > 50 anche dopo la log-trasformazione). Questo fenomeno può compromettere la stabilità numerica dei coefficienti OLS e rendere difficoltosa l’interpretazione individuale degli effetti.  

La log-trasformazione delle feature ha ridotto significativamente i VIF globali (ad es. landing_distance da ~172 a ~55, engine_power da ~24 a ~22, fuel_tank da ~27 a ~19), ma rimangono ancora variabili con VIF superiori a 10 (cruise_speed, length, fuel_tank, empty_weight).  

🔹 **Prossimi passi consigliati**  
1. **Rimuovere** o **combinare** le variabili con VIF estremamente alti (landing_distance vs empty_weight).  
2. Valutare la creazione di feature derivate (es. rapporto `power/weight`) per catturare l’informazione condivisa e ridurre la collinearità.  
3. Utilizzare Ridge o Lasso nel modello finale, che penalizzano automaticamente le variabili più collineari e stabilizzano la stima.  

Così avremo un modello più robusto e interpretabile, con coefficienti meno sensibili alle ridondanze tra le covariate.


## Feature engineering for VIF reduction

### ⚠️ Ora otteniamo **df** che è il nuovo dataframe uguale a logData a meno delle variabili con VIF > 10

In [ ]:
VIF_THRESHOLD = 10.0

# partiamo da una copia di logData
data_pruned = logData.copy()

# consideriamo solo le colonne numeriche
numeric_data_cleaned = (
    data_pruned
    .select_dtypes(include=[np.number])
    .replace([np.inf, -np.inf], np.nan)
    .dropna()
)

variables = numeric_data_cleaned.columns.tolist()
dropped_cols = []

def calculate_vif(df):
    df_const = sm.add_constant(df)
    vif = pd.DataFrame({
        "Regressor": df_const.columns,
        "VIF": [
            variance_inflation_factor(df_const.values, i)
            for i in range(df_const.shape[1])
        ]
    })
    return vif.sort_values("VIF", ascending=False)

while True:
    vif_df = calculate_vif(numeric_data_cleaned[variables])
    vif_no_const = vif_df[vif_df.Regressor != "const"]
    max_vif = vif_no_const["VIF"].max()
    if max_vif <= VIF_THRESHOLD:
        print(f"Tutti i VIF sono sotto {VIF_THRESHOLD}")
        break
    var_to_drop = vif_no_const.iloc[0]["Regressor"]
    print(f"Rimuovo {var_to_drop} (VIF={max_vif:.1f})")
    dropped_cols.append(var_to_drop)
    variables.remove(var_to_drop)
    numeric_data_cleaned.drop(columns=[var_to_drop], inplace=True)

print("Colonne eliminate:", dropped_cols)

# otteniamo df senza toccare data/logData
df = data_pruned.drop(columns=dropped_cols)
print("Colonne finali in df:", df.columns.tolist())


In [ ]:
#VIF finale
vif_final = calculate_vif(numeric_data_cleaned[variables])
print(vif_final)

#numeric_data

⚠️ **df** è il nuovo dataframe uguale a logData a meno delle variabili con VIF > 10.
usiamo **df** d'ora in avanti


In [ ]:
#heatmap    

numeric_data = df.select_dtypes(include=[np.number])


plt.figure(figsize=(12, 10))
sns.heatmap(
    numeric_data.corr(),
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    square=True,
    linewidths=.5,
    cbar_kws={"shrink": .75}
)
plt.title('Correlation Heatmap of Numeric Features')
plt.tight_layout()
plt.show()

In [ ]:
data.columns

In [ ]:
logData.columns

In [ ]:
df.columns

In [ ]:
#VIF
numeric_data = df.select_dtypes(include=[np.number])
# Handle missing or infinite values in the data
numeric_data_cleaned = numeric_data.replace([np.inf, -np.inf], np.nan).dropna()
numeric_data_with_const = sm.add_constant(numeric_data_cleaned)
# Calcola il VIF per ciascuna variabile
vif_data = pd.DataFrame()
vif_data["Regressor"] = numeric_data_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(numeric_data_with_const.values, i) for i in range(numeric_data_with_const.shape[1])]
vif_data = vif_data.sort_values("VIF", ascending=False)
print(vif_data)

In [ ]:
df['roc_mean'] = (df['all_eng_roc'] + df['out_eng_roc']) / 2
df['speed_margin'] = df['max_speed'] - df['stall_speed']
df['power_per_distance'] = df['engine_power'] / df['takeoff_distance']


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

X_new = df[['roc_mean', 'speed_margin', 'power_per_distance', 
            'wing_span', 'range']]   # price esclusa
X_new = add_constant(X_new)


vif = pd.DataFrame()
vif["Regressor"] = X_new.columns
vif["VIF"] = [variance_inflation_factor(X_new.values, i) for i in range(X_new.shape[1])]

print(vif)


In [ ]:
#nuovo df
df = df[['price', 'roc_mean', 'speed_margin', 'power_per_distance', 
          'wing_span', 'range']]  # price inclusa
print(df.columns)

In [ ]:
#heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(
    df.corr(),
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    square=True,
    linewidths=.5,
    cbar_kws={"shrink": .75}
)
plt.title('Correlation Heatmap of Numeric Features')
plt.tight_layout()
plt.show()

### Heatmap con correlazioni molto alte
# 💡 Passo indietro: stesse feaatures ma non log-trasformate 

### Creazione di nuove features 
- **roc_mean**: velocità di salita media, calcolata come media tra `all_eng_roc` e `out_eng_roc`.  
- **speed_margin**: margine di velocità operativo, ottenuto sottraendo la velocità di stallo (`stall_speed`) dalla velocità massima (`max_speed`).  
- **power_per_distance**: potenza erogata per metro di decollo, dato dal rapporto tra `engine_power` su `takeoff_distance`.  


In [ ]:
# Confronto con features non log‐trasformate
df2 = data[['all_eng_roc', 'out_eng_roc', 'max_speed', 'stall_speed',
            'engine_power', 'takeoff_distance', 'wing_span', 'range']].copy()

# nuove feature
df2['roc_mean'] = (df2['all_eng_roc'] + df2['out_eng_roc']) / 2
df2['speed_margin'] = df2['max_speed'] - df2['stall_speed']
df2['power_per_distance'] = df2['engine_power'] / df2['takeoff_distance']
df2['price'] = data['price']



## Rimozione delle features usate per la creazione di nuove features 

In [ ]:
# selezione finale
df2 = df2[['price', 'roc_mean', 'speed_margin', 'power_per_distance', 'wing_span', 'range']]

print(df2.columns.tolist())



In [ ]:
# heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(
    df2.corr(),
    annot=True, fmt='.2f',
    cmap='coolwarm',
    square=True,
    linewidths=.5,
    cbar_kws={"shrink": .75}
)
plt.title('Correlation Heatmap of Raw Selected Features')
plt.tight_layout()
plt.show()

In [ ]:
#VIF
numeric_data = df2.select_dtypes(include=[np.number])
# Handle missing or infinite values in the data
numeric_data_cleaned = numeric_data.replace([np.inf, -np.inf], np.nan).dropna()
numeric_data_with_const = sm.add_constant(numeric_data_cleaned)
# Calcola il VIF per ciascuna variabile
vif_data = pd.DataFrame()
vif_data["Regressor"] = numeric_data_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(numeric_data_with_const.values, i) for i in range(numeric_data_with_const.shape[1])]
vif_data = vif_data.sort_values("VIF", ascending=False)
print(vif_data)

## 💡 Usiamo **log-price**

In [ ]:
#ora uso df2 ma con log price
df2['price'] = np.log1p(df2['price'])
# Seleziona le colonne numeriche
numeric_data = df2.select_dtypes(include=[np.number])
# Handle missing or infinite values in the data
numeric_data_cleaned = numeric_data.replace([np.inf, -np.inf], np.nan).dropna()
numeric_data_with_const = sm.add_constant(numeric_data_cleaned)
# Calcola il VIF per ciascuna variabile
vif_data = pd.DataFrame()
vif_data["Regressor"] = numeric_data_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(numeric_data_with_const.values, i) for i in range(numeric_data_with_const.shape[1])]
vif_data = vif_data.sort_values("VIF", ascending=False)
print(vif_data)

In [ ]:
#heatmap di df2
plt.figure(figsize=(12, 10))
sns.heatmap(
    df2.corr(),
    annot=True, fmt='.2f',
    cmap='coolwarm',
    square=True,
    linewidths=.5,
    cbar_kws={"shrink": .75}
)
plt.title('Correlation Heatmap of Raw Selected Features (Log Transformed Price)')
plt.tight_layout()
plt.show()

## Conclusione
Dopo la skewness check log trasformando il dataset abbiamo ottenuto distribuzioni più simmetriche e gaussiane. purtroppo dopo la VIF check abbiamo notato che alcune variabili sono altamente collineari.
Nonostante la feature engineering la collinearità è rimasta alta.
### abbiamo deciso dunque di usare solo log price e il resto delle variabili senza log transformation.
Otteniamo dunque VIF piu bassi e una heatmap piu chiara con meno collinearità.

### All data are cleaned from na values 

### data

In [ ]:
#save as csv
data.to_csv("Data/aircraft_price_clean.csv", index=False)
data.columns

### logData

In [ ]:
#save as csv
logData.to_csv("Data/aircraft_price_cleaned_log.csv", index=False)
logData.columns

### After Feature Engineering and with log transformation

In [ ]:
#save as csv
df.to_csv("Data/aircraft_price_best-logEngineered.csv", index=False)
df.columns 

### After Feature Engineering and with out log transformation except for price

In [ ]:
#save as csv
df2.to_csv("Data/aircraft_price_Engineered.csv", index=False)
df2.columns 